# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
#import io
import pandas as pd

In [ ]:
url = "https://books.toscrape.com/"
header = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'}
#Hacer la petición a la url indicada
response = requests.get(url, headers = header, timeout = 3)
#comprobar la conexión de la petición 200 ok y 400, 500 bad
response.status_code
#imprimir la 500 primeras lineas para comprobar
response.text[:500]

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" /'

In [3]:
#crear el objeto `Beautifulsoup` 
soup = BeautifulSoup(response.content, "html.parser")

#examinando el documento html encuentro dentro de un div, un único <ul class='nav nav-list'> con <ul> anidado`y dentro
# una <a>`que contiene las categorias que busco
#usar .find para encontrar <ul>, el bloque principal de las categorias
nav_list = soup.find('ul', class_='nav nav-list')
#dentro encuentro 'ul' anidado
sub_list = nav_list.find('ul')
#y por último  localizo todas las <a> dentro del último <ul>
categoria = sub_list.find_all('a')



In [ ]:
#lista final con las categorías, libros y precios
libreria = []

#recorro todas las categorias y extraigo el texto
for i in categoria:
    nombre_categoria = str.strip(i.get_text())
    #completo la url relativa para cada página en 'categoria'
    url_categoria = url + i['href']
    #peticiones a las primeras págians de las categorias
    response = requests.get(url_categoria, headers=header)
    soup = BeautifulSoup(response.content, "html.parser")
    #extaer libros y precios
    #<article class="product_pod">
    libros = soup.find_all('article', class_='product_pod')
    
    for libro in libros:
        #dentro de 'article', class_='product_pod'busco etiqueta h3-->a para obtener el titulo del libro
        titulo = libro.h3.a['title']
        price = str.strip(libro.find('p', class_='price_color').get_text())#strip=tTrue
        libreria.append({"Categoria":nombre_categoria, "Título": titulo, "Precio":price[1:]})

#libreria

df = pd.DataFrame(libreria)

df.to_csv("libreria.csv")
df

,Categoria,Título,Precio
0,Travel,It's Only the Himalayas,45.17
1,Travel,Full Moon over Noah’s Ark: An Odyssey to Mount...,49.43
2,Travel,See America: A Celebration of Our National Par...,48.87
3,Travel,Vagabonding: An Uncommon Guide to the Art of L...,36.94
4,Travel,Under the Tuscan Sun,37.33
...,...,...,...
512,Politics,Why the Right Went Wrong: Conservatism--From G...,52.65
513,Politics,Equal Is Unfair: America's Misguided Fight Aga...,56.86
514,Cultural,Amid the Chaos,36.58
515,Erotica,Dark Notes,19.19
